In [1]:
import json 
from pathlib import Path
import shutil
import datetime as dt
from pprint import pprint

import pandas as pd

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.data_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


PROJECT_DIRECTORY = /Users/heslinge/Repos/Wattile/wattile


In [2]:
"""
This demo runs predictions on a pre-trained alpha model in ./tests/fixtures/alfa_exp_dir.
"""
exp_dir = PROJECT_DIRECTORY / "tests/fixtures/alfa_exp_dir"
with open(exp_dir / "configs.json", "r") as f:
    configs = json.load(f)

print(configs)

{'data_input': {'data_dir': 'tests/data/Synthetic Site', 'data_config': 'Synthetic Site Config.json', 'start_time': '2018-01-01T00:00:00-07:00', 'end_time': '2022-01-01T00:00:00-07:00', 'predictor_columns': ['Synthetic Weather Station Dew Point Temperature', 'Synthetic Weather Station Diffuse Horizontal Irradiance', 'Synthetic Weather Station Direct Normal Irradiance', 'Synthetic Weather Station Dry Bulb Temperature', 'Synthetic Weather Station Global Horizontal Irradiance', 'Synthetic Weather Station Relative Humidity', 'Synthetic Weather Station Wind Speed'], 'target_var': 'Synthetic Site Electricity Main Total Power'}, 'data_output': {'exp_dir': 'exp_dir', 'plot_comparison': True, 'plot_comparison_portion_start': 0.0, 'plot_comparison_portion_end': 1.0}, 'data_processing': {'feat_time': {'month_of_year': ['sincos'], 'day_of_week': ['binary_reg'], 'hour_of_day': ['sincos'], 'holidays': False}, 'resample': {'bin_interval': '15min', 'bin_closed': 'right', 'bin_label': 'right'}, 'feat_s

In [3]:
"""
Init the model with it's configs. Be sure to switch it's use case to prediction.
"""

configs["data_output"]["exp_dir"] = exp_dir
configs["learning_algorithm"]["use_case"] = "prediction"
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)

model

Logging to: exp_dir/output.out, PID: 13330


In [4]:
"""
If we want to get predictions for only one row, we have to find the time range needed to predict the nominal time.

we can use `get_input_window_for_output_time` to figure this out.
"""
nominal_time = pd.Timestamp(year=2020, month=1, day=1, tz=dt.timezone.utc)

start_time, end_time = model.get_input_window_for_output_time(nominal_time)
freq = model.configs["data_processing"]["resample"]["bin_interval"]
inclusive = model.configs["data_processing"]["resample"]["bin_label"]

predictors = model.configs["data_input"]["predictor_columns"]
target_var = model.configs["data_input"]["target_var"]

print(f"""
    To use this model to predict {target_var} for {nominal_time}, we will need to feed it data:
        - starting at {start_time}
        - ending at {end_time}
        - a rough frequency of {freq}
        - predictors named {predictors}
""")


    To use this model to predict Synthetic Site Electricity Main Total Power for 2020-01-01 00:00:00+00:00, we will need to feed it data:
        - starting at 2019-12-31 20:45:00+00:00
        - ending at 2020-01-01 00:00:00+00:00
        - a rough frequency of 15min
        - predictors named ['Synthetic Weather Station Dew Point Temperature', 'Synthetic Weather Station Diffuse Horizontal Irradiance', 'Synthetic Weather Station Direct Normal Irradiance', 'Synthetic Weather Station Dry Bulb Temperature', 'Synthetic Weather Station Global Horizontal Irradiance', 'Synthetic Weather Station Relative Humidity', 'Synthetic Weather Station Wind Speed']



In [5]:
"""
Great, now lets make some dummy data that matches those specs.
"""

data = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=freq, inclusive=inclusive))
for predictor in predictors:
    data[predictor] = data.index.hour * 100 + data.index.minute

# even though we are predicting, we must fill the target column with dummy data. It's not used though.
data[target_var] = data.index.hour * 100 + data.index.minute


data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
2019-12-31 21:00:00+00:00,2100,2100,2100,2100,2100,2100,2100,2100
2019-12-31 21:15:00+00:00,2115,2115,2115,2115,2115,2115,2115,2115
2019-12-31 21:30:00+00:00,2130,2130,2130,2130,2130,2130,2130,2130
2019-12-31 21:45:00+00:00,2145,2145,2145,2145,2145,2145,2145,2145
2019-12-31 22:00:00+00:00,2200,2200,2200,2200,2200,2200,2200,2200
2019-12-31 22:15:00+00:00,2215,2215,2215,2215,2215,2215,2215,2215
2019-12-31 22:30:00+00:00,2230,2230,2230,2230,2230,2230,2230,2230
2019-12-31 22:45:00+00:00,2245,2245,2245,2245,2245,2245,2245,2245
2019-12-31 23:00:00+00:00,2300,2300,2300,2300,2300,2300,2300,2300
2019-12-31 23:15:00+00:00,2315,2315,2315,2315,2315,2315,2315,2315


In [6]:
"""
Let's prep those for the model.
"""
train_df, val_df = prep_for_rnn(configs, data)
val_df

,Synthetic Weather Station Dew Point Temperature_min_lag12,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag12,Synthetic Weather Station Direct Normal Irradiance_min_lag12,Synthetic Weather Station Dry Bulb Temperature_min_lag12,Synthetic Weather Station Global Horizontal Irradiance_min_lag12,Synthetic Weather Station Relative Humidity_min_lag12,Synthetic Weather Station Wind Speed_min_lag12,sin_HOD_min_lag12,cos_HOD_min_lag12,DOW_binary_reg_0_min_lag12,...,DOW_binary_reg_0_mean,DOW_binary_reg_1_mean,DOW_binary_reg_2_mean,DOW_binary_reg_3_mean,DOW_binary_reg_4_mean,DOW_binary_reg_5_mean,DOW_binary_reg_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
2020-01-01 00:00:00+00:00,2100.0,2100.0,2100.0,2100.0,2100.0,2100.0,2100.0,-0.707107,0.707107,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.017213,0.999852,0


In [7]:
"""
One line in, one prediction (with many quantiles) out.
"""
results = model.predict(val_df)

results

FileNotFoundError: [Errno 2] No such file or directory: 'exp_dir/train_stats.json'